In [1]:
import numpy as np 

In [2]:
def quat2rot(q):
    R = np.array([[q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2, 2*(q[1] * q[2] - q[0] * q[3]),         2*(q[1] * q[3] + q[0] * q[2])],
                  [2*(q[1] * q[2] + q[0] * q[3]),         q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2, 2*(q[2] * q[3] - q[0] * q[1])],
                  [2*(q[1] * q[3] - q[0] * q[2]),         2*(q[2] * q[3] + q[0] * q[1]),         q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2]])
    return R
                  

def quat2zyz(q):
    r23 =  (q[2] * q[3] - q[1] * q[0])
    r13 =  (q[1] * q[3] + q[2] * q[0])
    r33 =  1 - 2*(q[1]**2 + q[2] ** 2)
    r31 =  (q[1] * q[3] - q[2] * q[0])
    r32 =  (q[2] * q[3] + q[1] * q[0])
    if r33 != 0:
        y = np.arccos(r33)
        x = np.arctan2(r32, r31)
        z = np.arctan2(r23, -r13)
        
    return np.array([x,y,z])

def quaternion_to_euler(q):
    (w, x, y, z) = (q[0], q[1], q[2], q[3])
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll = np.arctan2(t0, t1)
    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch = np.arcsin(t2)
    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw = np.arctan2(t3, t4)
    return np.array([roll, pitch, yaw])

def quatmult(x,y):
    return np.array([x[0]*y[0] - x[1]*y[1] - x[2]*y[2] - x[3]*y[3],
                     x[0]*y[1] + x[1]*y[0] + x[2]*y[3] - x[3]*y[2],
                     x[0]*y[2] - x[1]*y[3] + x[2]*y[0] + x[3]*y[1],
                     x[0]*y[3] + x[1]*y[2] - x[2]*y[1] + x[3]*y[0]])

def euler_to_quaternion(r):
    (yaw, pitch, roll) = (r[0], r[1], r[2])
    qz = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qx = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    return np.array([qw, qx, qy, qz])

In [3]:
def r_x(x): 
    return np.array([[1,0,0], [0, np.cos(x), - np.sin(x)], [0, np. sin(x), np.cos(x)]])
def r_y(x): 
    return np.array( [[np.cos(x), 0, np.sin(x)], [0,1,0], [-np. sin(x), 0 ,  np.cos(x)]])
def r_z(x): 
    return np.array([[np.cos(x), - np.sin(x), 0], [np. sin(x), np.cos(x),0], [0,0,1]])


def xyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), np.sin(x/2), 0, 0])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qz, quatmult(qy, qx))

def zyz2quat(x,y,z):
    qx = np.array([np.cos(x/2), 0, 0, np.sin(x/2)])
    qy = np.array([np.cos(y/2), 0, np.sin(y/2), 0])
    qz = np.array([np.cos(z/2), 0, 0, np.sin(z/2)])
    return quatmult(qx, quatmult(qy, qz))

def zyz2xyz(x,y,z):
    R = r_z(x) @ r_y(y) @ r_z(z)
    x = np.arctan(- R[1,2]/R[2,2])
    y = np.arcsin(R[0,2])
    z = np.arctan(-R[0,1] /R[0,0])
    return x,y,z

def xyz2zyz(e):
    R = r_x(e[0]) @ r_y(e[1]) @ r_z(e[2])
    if R[2,2] != 0:
        y = np.arccos(R[2,2])
        z = -np.arctan2(R[2,1], R[2,0]) +np.pi
        x = np.arctan2(R[1,2], R[0,2])
        
    return np.array([x,y,z])

In [4]:
# transforms points to quaterniions and back
x = 0.0
y = 0.3 
z = 1.3
xyz2quat(*zyz2xyz(x,y,z)) #- euler_to_quaternion(zyz2xyz(x,y,z))

array([ 0.78714464, -0.09043793,  0.11896528,  0.59839081])

In [10]:
MRP_H2R(np.array([1,0,0,0]))

array([0., 0., 0.])

In [8]:
#define transformation from H to R³
## works

def MRP_H2R(q):
    return 4 * q[1:]/(1+q[0])

def MRP_R2H(e):
    return 1/(16 + np.linalg.norm(e)**2) * np.array([16 - np.linalg.norm(e)**2, *8*e])

In [12]:
# complex conjugated 
def quat_cc(q):
    d = np.copy(q)
    d[1:] *= -1
    return d

In [13]:
x = 0
y = 0.5 
z = 1
a = euler_to_quaternion(zyz2xyz(x,y,z))

In [ ]:
quat2rot(a).T

In [ ]:
from numpy import sin, cos
def r_z_r_y_r_z(ret, v):
    ret[0,0] = -sin(v[0])*sin(v[2]) + cos(v[0])*cos(v[1])*cos(v[2])
    ret[0,1] = sin(v[0])*cos(v[2]) + sin(v[2])*cos(v[0])*cos(v[1])
    ret[0,2] = - sin(v[1])*cos(v[0])
    ret[1,0] = - sin(v[0])*cos(v[1])*cos(v[2]) - sin(v[2])*cos(v[0])
    ret[1,1] = -sin(v[0])*sin(v[2])*cos(v[1]) + cos(v[0])*cos(v[2])
    ret[1,2] = sin(v[0])*sin(v[1])
    ret[2,0] = sin(v[1])*cos(v[2])
    ret[2,1] = sin(v[1])*sin(v[2])
    ret[2,2] = cos(v[1])
    

In [ ]:
R = np.zeros((3,3))
r_z_r_y_r_z(R, [0,0.5,1])

In [ ]:
R

In [22]:
R = np.zeros((3,3))
r_z_r_y_r_z(R, [0,0.5,1])

In [23]:
R

array([[ 0.47415988,  0.73846026, -0.47942554],
       [-0.84147098,  0.54030231,  0.        ],
       [ 0.25903472,  0.40342268,  0.87758256]])

In [364]:
MRP_H2R(np.array([1,0,0,0]))

array([0., 0., 0.])

In [365]:
import sympy as s
def r_x(x): 
    return np.array([[1,0,0], [0, s.cos(x), - s.sin(x)], [0, s.sin(x), s.cos(x)]])
def r_y(x): 
    return np.array( [[s.cos(x), 0, s.sin(x)], [0,1,0], [-s.sin(x), 0 ,  s.cos(x)]])
def r_z(x): 
    return np.array([[s.cos(x), - s.sin(x), 0], [s.sin(x), s.cos(x),0], [0,0,1]])



In [367]:
x = s.Symbol('zyz[0]')
y = s.Symbol('zyz[1]')
z = s.Symbol('zyz[2]')
r_z(x) @ r_y(y) @ r_z(z)

array([[-sin(zyz[0])*sin(zyz[2]) + cos(zyz[0])*cos(zyz[1])*cos(zyz[2]),
        -sin(zyz[0])*cos(zyz[2]) - sin(zyz[2])*cos(zyz[0])*cos(zyz[1]),
        sin(zyz[1])*cos(zyz[0])],
       [sin(zyz[0])*cos(zyz[1])*cos(zyz[2]) + sin(zyz[2])*cos(zyz[0]),
        -sin(zyz[0])*sin(zyz[2])*cos(zyz[1]) + cos(zyz[0])*cos(zyz[2]),
        sin(zyz[0])*sin(zyz[1])],
       [-sin(zyz[1])*cos(zyz[2]), sin(zyz[1])*sin(zyz[2]), cos(zyz[1])]],
      dtype=object)

In [5]:
import bonndit.utilc.quaternions as qu

In [3]:
qu.MPR_H2R_q

Processing /home/johannes/UniBonn/forschung/bonndit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.24.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Created wheel for bonndit: filename=bonndit-0.2.0-cp38-cp38-linux_x86_64.whl size=30364533 sha256=7a0c9644efe60b23e2ce94f1e0704d8839cb3082e87357e05a9518724544732a
  Stored in directory: /tmp/pip-ephem-wheel-cache-elq6nu5z/wheels/2e/1e/92/44625976e847c3f0ac3a02bc8b14a2722cbba93db73d0e6491
Successfully built bonndit
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install .


Defaulting to user installation because normal site-packages is not writeable
Processing /home/xderes/bonndit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done